In [1]:
#alternative implementation of QAOA as mentioned in the paper

import numpy as np
import networkx as nx

from qiskit import BasicAer
from qiskit.aqua.algorithms import NumPyMinimumEigensolver
from qiskit.optimization.applications.ising import graph_partition
from qiskit.optimization.applications.ising.common import random_graph, sample_most_likely

from qiskit.aqua import aqua_globals
from qiskit.aqua.algorithms import QAOA
from qiskit.aqua.components.optimizers import COBYLA
from qiskit.circuit.library import TwoLocal

from qiskit.quantum_info import Pauli
from qiskit.aqua.operators import WeightedPauliOperator
from qiskit import IBMQ

In [2]:
#IBMQ.enable_account(token= , hub='ibm-q', group='open', project= 'main')

C:\Users\fege9\anaconda3\lib\site-packages\qiskit\providers\ibmq\ibmqfactory.py:109: UserWarning: Timestamps in IBMQ backend properties, jobs, and job results are all now in local time instead of UTC.
  warnings.warn('Timestamps in IBMQ backend properties, jobs, and job results '


<AccountProvider for IBMQ(hub='ibm-q', group='open', project='main')>

In [5]:
def qaoa2(n,B,C=None, backend='simulator'): 
    aqua_globals.random_seed = 10598

    provider = IBMQ.get_provider(hub='ibm-q') #getting the appropriate backend from IBM
    if backend=='melbourne':
        backend=provider.get_backend('ibmq_16_melbourne')
    elif backend=='simulator':
        backend=provider.get_backend('ibmq_qasm_simulator')
    
    optimizer = COBYLA()
    pauli_list = []

    for i in range(n):
        for j in range(i):
            if B[i, j] != 0:
                x_p = np.zeros(n, dtype=np.bool)
                z_p = np.zeros(n, dtype=np.bool)
                z_p[i] = True
                z_p[j] = True
                pauli_list.append([-B[i,j], Pauli(z_p, x_p)])
    
    quop=WeightedPauliOperator(paulis=pauli_list)
    
    
    qaoa = QAOA(quop, optimizer, quantum_instance=backend) #Alternative: BasicAer.get_backend('statevector_simulator')

    result = qaoa.compute_minimum_eigenvalue()

    x = sample_most_likely(result.eigenstate)
    

    return x

In [6]:
G=nx.barbell_graph(4,0)
n=8
B=nx.modularity_matrix(G)

result=qaoa2(n,B)
print(result)

[1 1 1 1 0 0 0 0]
